In [ ]:
!pip install -r requirements.txt

In [ ]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, AutoConfig, AutoModelForSeq2SeqLM
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
import os,torch
from huggingface_hub import HfFolder, login

In [ ]:
from fthelperutils import *
import pandas as pd

In [ ]:
ds = load_dataset("tarekziade/profanity-clean")

README.md:   0%|          | 0.00/337 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/91222 [00:00<?, ? examples/s]

In [ ]:
lgobj = intializelog()
dssplit = create_ds_splits(logger=lgobj,ds=ds,testsplitsize=0.4, seed=42, testvalidsplit=0.5)

INFO:root:Training dataset size: 54733
INFO:root:Validation dataset size: 18244
INFO:root:Test dataset size: 18245


In [ ]:
trainset = dssplit['train']
testset = dssplit['test']
valset = dssplit['validation']

In [ ]:
model_name = "google/flan-t5-base"


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = T5ForConditionalGeneration.from_pretrained(model_name)
model = model.to(device)
tokenizer = T5Tokenizer.from_pretrained(model_name)
# apply padding and truncation to ensure that all text have the same length (256 tokens).
def apply_tokenize(batch):
    return tokenizer(batch["text"], padding='max_length', truncation=True, max_length=256, return_tensors="pt")

train_ds = trainset.map(apply_tokenize, batched=True, batch_size=len(trainset))
val_ds = valset.map(apply_tokenize, batched=True, batch_size=len(valset))
test_ds = testset.map(apply_tokenize, batched=True, batch_size=len(testset))

In [ ]:
def apply_labels(example):
    # lbls should be sequence of token ids instead of integer #return_tensors="pt"
    lbl_text = "offensive" if example['is_offensive'] else "not offensive"
    example['labels'] = tokenizer(lbl_text,padding="max_length",truncation=True,max_length=256).input_ids
    return example

train_ds = train_ds.map(apply_labels)
val_ds = val_ds.map(apply_labels)
test_ds = test_ds.map(apply_labels)

In [ ]:
config = AutoConfig.from_pretrained(model_name)

In [ ]:
config

T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "

In [ ]:
# Parameters
L_RATE = 2e-4
BATCH_SIZE = 8
PER_DEVICE_EVAL_BATCH = 8
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 2
NUM_EPOCHS = 5
EVAL_STRATEGY = "epoch"
STEPSNUM = 500
# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="/content/resultstfivemdl",
   eval_strategy=EVAL_STRATEGY,
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   warmup_steps=STEPSNUM,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   save_strategy=EVAL_STRATEGY,
   load_best_model_at_end=True,
   predict_with_generate=True,
   push_to_hub=False,
   report_to="none"

)

trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=train_ds,
   eval_dataset=val_ds,
   processing_class=tokenizer

)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.000500,0.000530
2,0.000400,0.000388
3,0.000200,0.000442


Epoch,Training Loss,Validation Loss
1,0.000500,0.000530
2,0.000400,0.000388
3,0.000200,0.000442
4,0.000100,0.000609
5,0.000100,0.000801


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=34210, training_loss=0.07662503175019295, metrics={'train_runtime': 10031.2593, 'train_samples_per_second': 27.281, 'train_steps_per_second': 3.41, 'total_flos': 9.369706042884096e+16, 'train_loss': 0.07662503175019295, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.0003877593844663352,
 'eval_runtime': 185.0253,
 'eval_samples_per_second': 98.603,
 'eval_steps_per_second': 12.328,
 'epoch': 5.0}

In [ ]:
from checkinference import checkfinetunedflanmodel

In [ ]:
checkfinetunedflanmodel(inputtext = "You worst idiot")

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


enountered output: tensor([[    0, 12130,     1]])


'offensive'

In [ ]:
checkfinetunedflanmodel(inputtext = "You are asshole")

enountered output: tensor([[    0, 12130,     1]])


'offensive'

In [ ]:
checkfinetunedflanmodel(inputtext = "You are pretty")

enountered output: tensor([[    0,    59, 12130,     1]])


'not offensive'

In [ ]:
checkfinetunedflanmodel(inputtext = "You are great")

enountered output: tensor([[    0,    59, 12130,     1]])


'not offensive'

In [ ]:
checkfinetunedflanmodel(inputtext = "Have a nice day")

enountered output: tensor([[    0,    59, 12130,     1]])


'not offensive'

In [ ]:
checkfinetunedflanmodel(inputtext = "its been a great pleasure talking to you")

enountered output: tensor([[    0,    59, 12130,     1]])


'not offensive'

In [ ]:
checkfinetunedflanmodel(inputtext = "The best rates currently hover between 5% and 7% APY.")

enountered output: tensor([[    0,    59, 12130,     1]])


'not offensive'

In [86]:
checkfinetunedflanmodel(inputtext="hello")

enountered output: tensor([[    0,    59, 12130,     1]])


'not offensive'

In [85]:
checkfinetunedflanmodel(inputtext="you are a nasty person")

enountered output: tensor([[    0, 12130,     1]])


'offensive'

In [84]:
checkfinetunedflanmodel(inputtext="go to hell, you flimsy fellow")

enountered output: tensor([[    0, 12130,     1]])


'offensive'

In [83]:
checkfinetunedflanmodel(inputtext="I ll block Excirial He blocked me")

enountered output: tensor([[    0,    59, 12130,     1]])


'not offensive'

# Check scores for Test ds

In [ ]:
y_true = test_ds['is_offensive']
y_pred = trainer.predict(test_ds)

In [87]:
y_true[:10]

[0, 1, 0, 0, 0, 1, 0, 1, 0, 0]

In [ ]:
y_pred.predictions

array([[    0,    59, 12130, ...,     0,     0,     0],
       [    0, 12130,     1, ...,     0,     0,     0],
       [    0,    59, 12130, ...,     0,     0,     0],
       ...,
       [    0, 12130,     1, ...,     0,     0,     0],
       [    0, 12130,     1, ...,     0,     0,     0],
       [    0, 12130,     1, ...,     0,     0,     0]])

In [90]:
def calculate_metricsfrombin(y_true, y_pred):
    import warnings
    warnings.filterwarnings('always')
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    import numpy as np
    result = {}
    predicted = y_pred.predictions[:,0]
    averagesel = "weighted"
    # convert to binary prediction
    ypred_bin = np.where(predicted > 0, 1, 0)
    result['Accuracy'] = accuracy_score(y_true, ypred_bin)
    result['Precision'] = precision_score(y_true, ypred_bin, average=averagesel)
    result['Recall'] = recall_score(y_true, ypred_bin, average=averagesel)
    result['F1'] = f1_score(y_true, ypred_bin, average=averagesel)
    return result

In [92]:
calculate_metricsfrombin(y_true=y_true, y_pred=y_pred)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'Accuracy': 0.6790353521512743,
 'Precision': 0.46108900947120507,
 'Recall': 0.6790353521512743,
 'F1': 0.5492308531567629}

# Check scores for validation

In [ ]:
yv_true = val_ds['is_offensive']
yv_pred = trainer.predict(val_ds)

In [91]:
calculate_metricsfrombin(y_true=yv_true, y_pred=yv_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'Accuracy': 0.6732076299057225,
 'Precision': 0.45320851296328013,
 'Recall': 0.6732076299057225,
 'F1': 0.5417241768002412}

# **Check scores for Trainds**

In [95]:
import warnings
warnings.filterwarnings('ignore')

yt_true = train_ds['is_offensive']
yt_pred = trainer.predict(train_ds)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Streaming output truncated to the last 5000 lines.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Tra

In [96]:
calculate_metricsfrombin(y_true=yt_true, y_pred=yt_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'Accuracy': 0.6674949299325819,
 'Precision': 0.44554948148570234,
 'Recall': 0.6674949299325819,
 'F1': 0.5343938065271553}